In [4]:
pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [41]:
import os
from os import environ
from dotenv import load_dotenv
load_dotenv()

True

In [42]:
# Load credentials
os.environ["AZURE_OPENAI_KEY"] = os.getenv('AZURE_OPENAI_KEY')
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv('AZURE_OPENAI_ENDPOINT')
os.environ["AZURE_OPENAI_API_VERSION"] = os.getenv('AZURE_OPENAI_API_VERSION')
os.environ["AZURE_OPENAI_DEPLOYMENT"] = os.getenv('AZURE_OPENAI_DEPLOYMENT')
os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT"] = os.getenv('AZURE_OPENAI_EMBEDDING_DEPLOYMENT')

# LCEL and Pipe Operator

## Pipe Operator

The `|` symbol chains together the different components from the output from one component as input into the next component.

## Runnable Interface

1. `invoke`: stream-back chunks of the response
2. `stream`: call the chain on an input
3. `batch`: call the chain on a list of inputs

Corresponding async methods:
1. `ainvoke`: stream back chunks of the response async
2. `astream`: call the chain on an input async
3. `abatch`: call the chain on a list of inputs async

## Input & Output Schemas


In [43]:
from langchain_openai import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

In [44]:
# Load Azure OpenAI model
model = AzureChatOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    temperature=0
)

In [45]:
# Generate Prompt and Chain
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
chain = prompt | model | StrOutputParser()

In [10]:
chain.input_schema.schema()

{'properties': {'topic': {'title': 'Topic', 'type': 'string'}},
 'required': ['topic'],
 'title': 'PromptInput',
 'type': 'object'}

In [11]:
prompt.input_schema.schema()

{'properties': {'topic': {'title': 'Topic', 'type': 'string'}},
 'required': ['topic'],
 'title': 'PromptInput',
 'type': 'object'}

In [12]:
model.input_schema.schema()

{'$defs': {'AIMessage': {'additionalProperties': True,
   'description': 'Message from an AI.\n\nAIMessage is returned from a chat model as a response to a prompt.\n\nThis message represents the output of the model and consists of both\nthe raw output as returned by the model together standardized fields\n(e.g., tool calls, usage metadata) added by the LangChain framework.',
   'properties': {'content': {'anyOf': [{'type': 'string'},
      {'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]},
       'type': 'array'}],
     'title': 'Content'},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'response_metadata': {'title': 'Response Metadata', 'type': 'object'},
    'type': {'const': 'ai',
     'default': 'ai',
     'enum': ['ai'],
     'title': 'Type',
     'type': 'string'},
    'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'title': 'Name'},
    'id': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'd

In [46]:
chain.invoke({"topic": "Programming"})

'Why do programmers prefer dark mode?\n\nBecause light attracts bugs!'

In [47]:
chain.batch([
    {"topic": "Data Analyst"},
    {"topic": "Data Science"}
])

['Why did the data analyst bring a ladder to work?\n\nBecause they heard the data was in the cloud!',
 "Why did the data scientist break up with the statistician?\n\nBecause they couldn't find any common variance!"]

In [49]:
import time
for s in chain.stream({"topic": "Bank"}):

    time.sleep(0.1)  # Sleep for 0.1 seconds

    print(s, end="", flush=True)

Sure, here's a bank-related joke for you:

Why did the banker switch careers?

He lost interest!

## Add Stop Sequence

In [50]:
chain = prompt | model.bind(stop="\n")
chain.invoke({"topic": "Machine Learning"})

AIMessage(content='Why did the neural network go to therapy?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 14, 'total_tokens': 23, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_67802d9a6d', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-c239c8ed-f11c-4624-aee3-fdd72a27cbae-0', usage_metadata={'input_tokens': 14, 'output_tokens': 9, 'total_tokens': 23, 'input_token_details': {}, 'output_token_de

# LCEL with RAG

## Setup Vectorstore

In [51]:
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from operator import itemgetter

In [52]:

# Initialize Azure OpenAI Embeddings
embeddings = AzureOpenAIEmbeddings(
    azure_deployment=os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    openai_api_key=os.environ["AZURE_OPENAI_KEY"],
    chunk_size=100
)

# Create a more diverse and informative vector store
texts = [
    "The Eiffel Tower in Paris is 324 meters tall",
    "Photosynthesis is the process by which plants convert light into energy",
    "The human genome contains approximately 3 billion base pairs",
    "Machine learning is a subset of artificial intelligence",
    "The Great Barrier Reef is the world's largest coral reef system",
    "Quantum computing leverages principles of quantum mechanics for computation",
    "The Mona Lisa was painted by Leonardo da Vinci in the early 16th century",
    "Climate change is causing global temperatures to rise at an unprecedented rate"
]

vectorstore = FAISS.from_texts(texts, embedding=embeddings)

retriever = vectorstore.as_retriever()

In [53]:
template = """Answer the question based only on the following context:

{context}

Question: {question}"""

prompt = ChatPromptTemplate.from_template(template)
# use previous model
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [54]:
chain.invoke("What kind of music does Ridhwan like?")

'The provided context does not contain any information about the kind of music Ridhwan likes.'

In [55]:
chain.invoke("What is Mona Lisa")

'The Mona Lisa is a painting by Leonardo da Vinci created in the early 16th century.'

In [56]:
template_2 = """Answer the question based only on the following context:
{context}

Question: {question}

Answer like a {character_type}."""

prompt_2 = ChatPromptTemplate.from_template(template_2)

chain_2 = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "character_type": itemgetter("character_type")
    }
    | prompt_2
    | model
    | StrOutputParser()
)

In [57]:
chain_2.invoke({"question": "What is Eiffel Tower?", "character_type": "pirate"})

"Arrr, the Eiffel Tower be a grand structure in Paris standin' 324 meters tall!"